In [36]:
%pip install google-cloud-bigquery
%pip install "google-cloud-bigquery[pandas]"
%pip install db-dtypes

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [37]:
from google.cloud import bigquery
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/tanya/Desktop/Data Science Journey/SQL/spheric-passkey-475414-m3-613464412c9f.json"

In [38]:
#we need to create a client object to retrieve information from BigQuery datasets
client = bigquery.Client(project="spheric-passkey-475414-m3")  # ← your project ID

In [39]:
# datasets exist in projects in bigquery. we would like to work with hacker_news dataset contained in bigquery-public-data project
# here we connect to the dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")


# now let's fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [40]:
# this dataset is like a set of tables. so now let's list those tables 
tables = list(client.list_tables(dataset))

# now we want the names of those tables. each of them has an id and this code shows the names of the tables
for table in tables:
    print(table.table_id)

# there is one table!

full


In [41]:
# Let's open a table now by calling it
# first we construct a reference to the table
table_ref = dataset_ref.table("full")

# then through that reference we fetch the table
table = client.get_table(table_ref)

In [42]:
# let's investigate the table. 
# print information on all the columns

table.schema

#each schemafield holds into about a specific column. 
# first is column name
# second is data type 
# third os mode of the column (nullable means allows null values)
# fourth is description of the data of the column 

[SchemaField('title', 'STRING', 'NULLABLE', None, 'Story title', (), None),
 SchemaField('url', 'STRING', 'NULLABLE', None, 'Story url', (), None),
 SchemaField('text', 'STRING', 'NULLABLE', None, 'Story or comment text', (), None),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', None, 'Is dead?', (), None),
 SchemaField('by', 'STRING', 'NULLABLE', None, "The username of the item's author.", (), None),
 SchemaField('score', 'INTEGER', 'NULLABLE', None, 'Story score', (), None),
 SchemaField('time', 'INTEGER', 'NULLABLE', None, 'Unix time', (), None),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', None, 'Timestamp for the unix time', (), None),
 SchemaField('type', 'STRING', 'NULLABLE', None, 'type of details (comment comment_ranking poll story job pollopt)', (), None),
 SchemaField('id', 'INTEGER', 'NULLABLE', None, "The item's unique id.", (), None),
 SchemaField('parent', 'INTEGER', 'NULLABLE', None, 'Parent comment ID', (), None),
 SchemaField('descendants', 'INTEGER', 'NULLABLE', N

In [43]:
# let's have a look at the first five rows of the full table

client.list_rows(table, max_results = 5).to_dataframe()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,None,<NA>,None,<NA>,1437718014,2015-07-24 06:06:54+00:00,job,9940325,<NA>,<NA>,<NA>,<NA>
1,None,None,None,<NA>,None,<NA>,1437719661,2015-07-24 06:34:21+00:00,story,9940389,<NA>,<NA>,<NA>,<NA>
2,None,None,None,<NA>,None,<NA>,1437719725,2015-07-24 06:35:25+00:00,story,9940392,<NA>,<NA>,<NA>,<NA>
3,None,None,None,True,None,<NA>,1260795502,2009-12-14 12:58:22+00:00,story,994049,<NA>,<NA>,<NA>,<NA>
4,None,None,None,<NA>,None,<NA>,1260795824,2009-12-14 13:03:44+00:00,story,994058,<NA>,<NA>,<NA>,<NA>


In [44]:
# the method above also lets us to have information of a specific column
client.list_rows(table, selected_fields = table.schema[:1], max_results = 5).to_dataframe()

,title
0,None
1,None
2,None
3,None
4,None


Group by, Count

Which Hacker News comments generated the most discussion?

In [45]:
#parent is the column where there are comments that have been replied to

query_popular = """
            select parent, count(id)
            from `bigquery-public-data.hacker_news.full`
            group by parent 
            having count(id) > 10
            """

In [46]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)

query_job = client.query(query_popular, job_config=safe_config)

popular_comments = query_job.to_dataframe()

popular_comments.head()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,parent,f0_
0,9269660,51
1,9392780,102
2,9703384,52
3,9738717,55
4,9775897,41


Aliasing

In [47]:
# count(1) just counts rows in each group. it does not focus on any particular column so it is computationally more efficient

query_popular_improved = """
                        select parent, count(1) as NumPosts
                        from `bigquery-public-data.hacker_news.full`
                        group by parent
                        having count(1) > 10
                        """

In [48]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)

query_job = client.query(query_popular_improved, job_config=safe_config)

popular_comments_improved = query_job.to_dataframe()

popular_comments_improved.head()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,parent,NumPosts
0,8894168,50
1,6996064,45
2,6998936,51
3,34455854,47
4,34484986,54


Let's work with OpenAQ dataset 

In [49]:
dataset_ref_aq = client.dataset("openaq", project = "bigquery-public-data") #build a reference to the dataset

dataset_aq = client.get_dataset(dataset_ref_aq)  #fetch the dataset

tables_aq = list(client.list_tables(dataset_aq))  #list all the tables in openaq dataset

tabs = []
for table in tables_aq: #print the names of the tables
    tabs.append(table.table_id)

print(f"There are {len(tabs)} tables, and their names are: {tabs}")
    

There are 1 tables, and their names are: ['global_air_quality']


Now let's return the table columns and rows 

In [50]:
# first let's create a table reference
table_ref_aq = dataset_ref_aq.table("global_air_quality")

# fetch the table through reference of the table
table_aq_gaq = client.get_table(table_ref_aq)

#now let's list the rows
client.list_rows(table_aq_gaq, max_results=5).to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours,location_geom
0,"Borówiec, ul. Drapałka",Borówiec,PL,bc,0.85217,2022-04-28 07:00:00+00:00,µg/m³,GIOS,1.0,52.276794,17.074114,POINT(52.276794 1)
1,"Kraków, ul. Bulwarowa",Kraków,PL,bc,0.91284,2022-04-27 23:00:00+00:00,µg/m³,GIOS,1.0,50.069308,20.053492,POINT(50.069308 1)
2,"Płock, ul. Reja",Płock,PL,bc,1.41000,2022-03-30 04:00:00+00:00,µg/m³,GIOS,1.0,52.550938,19.709791,POINT(52.550938 1)
3,"Elbląg, ul. Bażyńskiego",Elbląg,PL,bc,0.33607,2022-05-03 13:00:00+00:00,µg/m³,GIOS,1.0,54.167847,19.410942,POINT(54.167847 1)
4,"Piastów, ul. Pułaskiego",Piastów,PL,bc,0.51000,2022-05-11 05:00:00+00:00,µg/m³,GIOS,1.0,52.191728,20.837489,POINT(52.191728 1)


let's fetch the cities in the united states (where)

In [51]:
#let's write our query
query = """
        SELECT city
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

# let's add more columns 
query2 = """
        select city, country
        from `bigquery-public-data.openaq.global_air_quality`
        where country = 'US'
        """

In [52]:
# We need to use the query to return what we want 
query_job = client.query(query2)

# query_job is the subset of the data we filtered and now we turn it into a pandas dataframe called us_cities
us_cities = query_job.to_dataframe()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [53]:
# let's get the value counts of the dataframe
us_cities.city.value_counts().head()

city
Phoenix-Mesa-Scottsdale                     39414
Los Angeles-Long Beach-Santa Ana            27479
Riverside-San Bernardino-Ontario            26887
New York-Northern New Jersey-Long Island    25417
San Francisco-Oakland-Fremont               22710
Name: count, dtype: int64

In [54]:
us_cities

,city,country
0,HOWARD,US
1,HOWARD,US
2,HOWARD,US
3,HOWARD,US
4,HOWARD,US
...,...,...
1421346,Washington-Arlington-Alexandria,US
1421347,Washington-Arlington-Alexandria,US
1421348,Washington-Arlington-Alexandria,US
1421349,Washington-Arlington-Alexandria,US


Date format: YYYY-MM-DD <Br>
Datetime format: YYYY-MM-DD HH:MI:SS

EXTRACT

In [67]:
import pandas as pd

df_list = [[1, "Dr.Harris Bonkers", "Rabbit", "2019-04-18"], [4, "Tom", "Cat", "2019-05-16"], 
           [2, "Moon", "Dog", "2019-01-07"], [3, "Ripley", "Cat", "2019-02-23"]]

df = pd.DataFrame(df_list, columns=['ID', "Name", "Animal", "Date"])
df["Date"] = pd.to_datetime(df["Date"])
df.style.hide()      #hides the automatic indexing 

ID,Name,Animal,Date
1,Dr.Harris Bonkers,Rabbit,2019-04-18 00:00:00
4,Tom,Cat,2019-05-16 00:00:00
2,Moon,Dog,2019-01-07 00:00:00
3,Ripley,Cat,2019-02-23 00:00:00


In [63]:
%pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 6.4 MB/s  0:00:02 eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import duckdb

# Extracts day of the date and creates a column for those values
anm_query = """
            select Name, Extract(day from Date) as Day
            from df
            """

result = duckdb.query(anm_query).to_df()
print(result)

                Name  Day
0  Dr.Harris Bonkers   18
1                Tom   16
2               Moon    7
3             Ripley   23


In [ ]:
# Extracts week of that date (which is not part of the cell) so it can find which week of the year that is 
anm_query2 = """
            select Name, Extract(week from Date) as Week
            from df
            """

result2 = duckdb.query(anm_query2).to_df()
print(result2)

                Name  Week
0  Dr.Harris Bonkers    16
1                Tom    20
2               Moon     2
3             Ripley     8


Now we would like to tackle the US Traffic Fatality Records database that contains information on traffic accidents in the US where at least one person died.

In [70]:
traffic_ref = client.dataset("nhtsa_traffic_fatalities", project="bigquery-public-data")

traffic_dataset = client.get_dataset(traffic_ref)

traffic_tables = list(client.list_tables(traffic_dataset)) 

tabs = []
for table in traffic_tables: 
    tabs.append(table.table_id)

print(f"There are {len(tabs)} tables, and their names are: {tabs}")
    

There are 143 tables, and their names are: [' accident_2015', ' accident_2016', ' accident_2017', ' accident_2018', ' accident_2019', ' accident_2020', ' cevent_2015', ' cevent_2016', ' cevent_2017', ' cevent_2018', ' cevent_2019', ' cevent_2020', ' damage_2015', ' damage_2016', ' damage_2017', ' damage_2018', ' damage_2019', ' damage_2020', ' distract_2015', ' distract_2016', ' distract_2017', ' distract_2018', ' distract_2019', ' distract_2020', ' drimpair_2015', ' drimpair_2016', ' drimpair_2017', ' drimpair_2018', ' drimpair_2019', ' drimpair_2020', ' factor_2015', ' factor_2016', ' factor_2017', ' factor_2018', ' factor_2019', ' factor_2020', ' maneuver_2015', ' maneuver_2016', ' maneuver_2017', ' maneuver_2018', ' maneuver_2019', ' maneuver_2020', ' nmcrash_2015', ' nmcrash_2016', ' nmcrash_2017', ' nmcrash_2018', ' nmcrash_2019', ' nmcrash_2020', ' nmimpair_2015', ' nmimpair_2016', ' nmimpair_2017', ' nmimpair_2018', ' nmimpair_2019', ' nmimpair_2020', ' nmprior_2015', ' nmprior

In [71]:
table2015_ref = traffic_dataset.table("accident_2015")

table2015 = client.get_table(table2015_ref)

client.list_rows(table2015, max_results=5).to_dataframe()

,state_number,state_name,consecutive_number,number_of_vehicle_forms_submitted_all,number_of_motor_vehicles_in_transport_mvit,number_of_parked_working_vehicles,number_of_forms_submitted_for_persons_not_in_motor_vehicles,number_of_persons_not_in_motor_vehicles_in_transport_mvit,number_of_persons_in_motor_vehicles_in_transport_mvit,number_of_forms_submitted_for_persons_in_motor_vehicles,...,minute_of_ems_arrival_at_hospital,related_factors_crash_level_1,related_factors_crash_level_1_name,related_factors_crash_level_2,related_factors_crash_level_2_name,related_factors_crash_level_3,related_factors_crash_level_3_name,number_of_fatalities,number_of_drunk_drivers,timestamp_of_crash
0,30,Montana,300019,5,5,0,0,0,7,7,...,45,0,None,0,None,0,None,1,0,2015-03-28 14:58:00+00:00
1,39,Ohio,390099,7,7,0,0,0,15,15,...,24,27,Backup Due to Prior Crash,0,None,0,None,1,0,2015-02-14 11:19:00+00:00
2,49,Utah,490123,16,16,0,0,0,28,28,...,99,0,None,0,None,0,None,1,0,2015-04-14 12:24:00+00:00
3,48,Texas,481184,6,5,1,0,5,5,10,...,99,0,None,0,None,0,None,1,0,2015-05-27 16:40:00+00:00
4,41,Oregon,410333,11,11,0,0,0,14,14,...,99,0,None,0,None,0,None,1,0,2015-11-17 18:17:00+00:00


We would like to see which day of the week has the most fatal motor accidents

In [86]:
query3 = """
        select extract(dayofweek from timestamp_of_crash) as Day_of_week , count(consecutive_number) as Num_accidents
        from `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
        group by Day_of_week
        order by Num_accidents Desc
        """


safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)

query_job = client.query(query3, job_config=safe_config)

accident_by_day = query_job.to_dataframe()

accident_by_day.head()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,Day_of_week,Num_accidents
0,7,5659
1,1,5298
2,6,4916
3,5,4460
4,4,4182


##### In Bigquery, the dayofweek returns an integer from 1 to 7:<BR>
1: Sunday <BR>
2: Monday<BR>
3: Tuesday<BR>
4: Wednesday<BR>
5: Thursday<BR>
6: Friday<BR>
7: Saturday